In [1]:
import pandas as pd
import numpy as np
import os
import django
from django.db.models import Max, Min, Avg, Q, F
from asgiref.sync import sync_to_async
import tqdm
from collections import defaultdict
from pandarallel import pandarallel
import requests
import json

import ctypes
from ctypes import c_char_p, cdll
GoInt64 = ctypes.c_int64
GoInt = GoInt64
archive_node = "http://localhost:19545"

from etherscan.utils.parsing import ResponseParser as parser
pandarallel.initialize(progress_bar=True)
# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
# os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# django.setup()

from debtmonitor.models import *

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
token_dict = dict(
    usdc = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    usdt = '0xdac17f958d2ee523a2206206994597c13d831ec7',
    dai =  '0x6b175474e89094c44da98b954eedeac495271d0f',
    # common collateral asset
    weth = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"
)
decimal_dict = dict(
    usdc = 18,
    usdt = '0xdac17f958d2ee523a2206206994597c13d831ec7',
    dai =  '0x6b175474e89094c44da98b954eedeac495271d0f',
    # common collateral asset
    weth = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"
)
revert_token_dict = {v: k for k, v in token_dict.items()}

In [3]:
@sync_to_async
def get_potential_target():
    data = pd.DataFrame(
        list(
            LendingPoolInteraction.objects.all().values()
        )
    )
    return data['on_behalf_of'].unique().tolist()

@sync_to_async
def get_interaction_data(target):
    data = pd.DataFrame(
        list(
            LendingPoolInteraction.objects.filter(
                on_behalf_of=target
            ).annotate(
                 block_num=F('block_number__number'),
            ).all().values()
        )
    )
    return data

@sync_to_async
def get_reserves_status():
    data = pd.DataFrame(
        list(
            ReservesStatus.objects.annotate(
                 block_num=F('block_number__number'),
            ).all().values()
        )
    )
    return data

@sync_to_async
def get_liquidation_call(target=None):
    if target is None:
        data = pd.DataFrame(
            list(
                LiquidationCall.objects.annotate(
                    block_num=F('block_number__number'),
                ).all().values()
            )
        )
    else:
        data = pd.DataFrame(
            list(
                LiquidationCall.objects.filter(
                    on_behalf_of=target
                ).annotate(
                    block_num=F('block_number__number'),
                ).all().values()
            )
        )
    return data


In [4]:
ttt = await get_liquidation_call()
ttt

,id,block_number_id,index,on_behalf_of,collateral_asset,debt_asset,debt_to_cover,liquidated_collateral_amount,liquidator,receive_atoken,transaction_hash,block_num
0,1,1638825,51,0x26fc9fb5aab707e49d1ccbb6260ef103614ecdd5,0x408e41876cccdc0f92210600ef50372656052a38,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3.015000e+17,2.262690e+21,0x00000000b2ff98680adaf8a3e382176bbfc34c8f,False,,12982031
1,2,1638825,64,0x272bb028b2d59e3ef0023d6fb3bcd35f573b714a,0x514910771af9ca656af840dff83e8264ecf986ca,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2.500000e+17,3.548319e+19,0x00000000b2ff98680adaf8a3e382176bbfc34c8f,False,,12982031
2,3,949256,94,0x72dcfb1b13267fb2f06bfd3d0075b07d4d7811ed,0x8798249c2e607446efb7ad49ec89dd1865ff4272,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,7.537820e+17,2.344570e+20,0x19256c009781bc2d1545db745af6dfd30c7e9cfa,False,,12984312
3,4,949387,76,0xe711f3cc27eb2c36b61f25951d8871d1848a5149,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xdac17f958d2ee523a2206206994597c13d831ec7,1.586725e+09,5.430224e+17,0xb4ed60934f8637f6c57c80610d92f97d567d249d,False,,12985028
4,5,39666,600,0xaa2062697fbf9f65cf685308d2c0019b028d7bd6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3.732456e+09,1.310691e+18,0x19256c009781bc2d1545db745af6dfd30c7e9cfa,False,,12985729
...,...,...,...,...,...,...,...,...,...,...,...,...
17249,17250,2125958,56,0x0a54adf196073e673e59e7d96c4e8561fbce4a20,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3.863125e+07,2.622528e+16,0xb206ebe579be55f5b57119bb2e7cc63708eda1aa,False,,15178767
17250,17251,1976800,68,0xc8500012787aa5de7015b3a7ee38060179550ac5,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2.547968e+07,1.729718e+16,0xb206ebe579be55f5b57119bb2e7cc63708eda1aa,False,,15178825
17251,17252,1976800,86,0xc8500012787aa5de7015b3a7ee38060179550ac5,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,1.273984e+07,8.648592e+15,0x2005bf3d68bd245aac41c016da92c7d0b0caa1d6,True,,15178825
17252,17253,1976808,66,0xc8500012787aa5de7015b3a7ee38060179550ac5,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,6.369921e+06,4.324296e+15,0x2005bf3d68bd245aac41c016da92c7d0b0caa1d6,True,,15178849


In [5]:
# log:
block_number = 14655687
index = 43


api_key = 'JSPD2IG21CPF9PHIKQP4IEW9R8KN1NJSYH'

url = f"https://api.etherscan.io/api?module=proxy\
&action=eth_getTransactionByBlockNumberAndIndex\
&tag={hex(block_number)}\
&index={hex(index)}\
&apikey={api_key}"

r = requests.get(url, headers={"User-Agent": ""})
df = pd.DataFrame(parser.parse(r).items(), columns=['key', 'value'])
df.set_index('key', inplace=True)
df

,value
key,
blockHash,0xdc69d208ffb9758a97b8a5a6bcff9682766c2ff40e5e...
blockNumber,0xdfa0c7
from,0x3b794929566e3ba0f25e4263e1987828b5c87161
gas,0xc350
gasPrice,0x1a944cfab4
hash,0xdba8e0e5a5603c5dffb452f1f05d613942a2f253db56...
input,0x
nonce,0x5dd1b
to,0x987a3a7e99ee7d463c2fbddc8e0c1a6280bb1e4f


In [6]:

url = f"https://api.etherscan.io/api\
?module=logs\
&action=getLogs\
&fromBlock={block_number}\
&toBlock={block_number}\
&address=0x7d2768dE32b0b80b7a3454c06BdAc94A69DDc7A9\
&topic0=0xe413a321e8681d831f4dbccbca790d2952b56f977908e45be37335533e005286\
&apikey={api_key}"

r = requests.get(url, headers={"User-Agent": ""})
log_list = parser.parse(r)
for log in log_list:
    if log['logIndex'] == hex(index):
        break
df = pd.DataFrame(log.items(), columns=['key', 'value'])
df.set_index('key', inplace=True)
print(df.loc['transactionHash'].values[0])
df

0xa5679867e5479d40efd207463b565eea622290a8b52b68e7ee9f8597c0fa9518


,value
key,
address,0x7d2768de32b0b80b7a3454c06bdac94a69ddc7a9
topics,[0xe413a321e8681d831f4dbccbca790d2952b56f97790...
data,0x00000000000000000000000000000000000000000000...
blockNumber,0xdfa0c7
blockHash,0xdc69d208ffb9758a97b8a5a6bcff9682766c2ff40e5e...
timeStamp,0x6266f812
gasPrice,0x16f8a6e3cf
gasUsed,0xc020a
logIndex,0x2b


In [7]:
read_from_csv = True

if not read_from_csv:

    # targets = await get_potential_target()
    liquidation_call_df = await get_liquidation_call()
    # liquidation_call_
    targets = set(liquidation_call_df['on_behalf_of'].to_list())
    # targets = list(set(liquidation_call_targets).intersection(set(targets)))

    potential_targets = []
    targets_amount = np.inf
    other_token_tolerance_amount = 0

    for target in tqdm.tqdm(targets):
        interaction_df = await get_interaction_data(target)
        interaction_df = interaction_df[interaction_df['action'] != "LiquidationCall"]
        if interaction_df.shape[0] == 0: continue
        potential = True
        other_token_counter = 0
        for token in set(interaction_df['reserve'].to_list()):
            if token not in token_dict.values():
                other_token_counter += 1
                potential = False
                # break
        if potential or other_token_counter <= other_token_tolerance_amount: 
            potential_targets.append(target)
            # print(target)
            if len(potential_targets) >= targets_amount:
                break
    potential_targets = pd.DataFrame(potential_targets, columns=['on_behalf_of'])
    potential_targets.to_csv("../data/potential_targets.csv")
    # potential_targets
else:
    potential_targets = pd.read_csv("../data/potential_targets.csv", index_col=0)
potential_targets

,on_behalf_of
0,0x3dd3d4db709ddf8b18d05e1ca868b141aa7962e5
1,0x016a42b47b051be2946a35fb72774e00a7b8e820
2,0xf8581a99ad0a2f63e1e8cbeaeb6b31d6585824cc
3,0xe5423d0f7a99ea90b76f972bbfe0d8f585395acb
4,0x39057008ca34c07eefa5bfe78a27c46db2bd49a2
...,...
1361,0x17e062e6cdecd51fdb61f99689c86eb8b71f8b40
1362,0xe10c03212e262b0eb1c1652f0bc0bd7c69cc4d00
1363,0x2330b2aaa7d8277df874b35ff85682f4576011ef
1364,0x569dc59033712942db4f907c51b5e9403f12c817


In [99]:
target_address = "0x39057008ca34c07eefa5bfe78a27c46db2bd49a2"

In [100]:
liquidation_df = await get_liquidation_call(target_address)
liquidation_df

,id,block_number_id,index,on_behalf_of,collateral_asset,debt_asset,debt_to_cover,liquidated_collateral_amount,liquidator,receive_atoken,transaction_hash,block_num
0,16582,2035240,45,0x39057008ca34c07eefa5bfe78a27c46db2bd49a2,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xdac17f958d2ee523a2206206994597c13d831ec7,167891534.0,1.502557e+17,0x8472ee188899569cf6ef42bb2038facae66972d2,False,,15094461


In [101]:
ttt = await get_interaction_data(target_address)
ttt.sort_values("block_num")

,id,block_number_id,index,on_behalf_of,action,reserve,amount,rate_mode,rate,transaction_hash,block_num
0,155785,1142565,122,0x39057008ca34c07eefa5bfe78a27c46db2bd49a2,Deposit,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1.500000e+17,-1,-1.000000e+00,,13883845
1,158796,1143078,460,0x39057008ca34c07eefa5bfe78a27c46db2bd49a2,Borrow,0xdac17f958d2ee523a2206206994597c13d831ec7,5.035837e+08,2,3.423295e+25,,13885135
2,376109,2035240,45,0x39057008ca34c07eefa5bfe78a27c46db2bd49a2,LiquidationCall,-1,-1.000000e+00,-1,-1.000000e+00,,15094461


In [102]:
await get_reserves_status()

CancelledError: 

In [103]:
interaction_df = await get_interaction_data(target_address)
reserves_status = await get_reserves_status()
liquidation_df = await get_liquidation_call(target_address)

# potential = True
# for token in set(interaction_df['reserve'].to_list()):
#     if token not in token_dict.values():
#         potential = False
#         break
# assert potential, "Only support WETH DAI USDT and USDC"

interaction_df = interaction_df['action block_num index on_behalf_of reserve amount rate_mode rate'.split(' ')].copy()
reserves_status = reserves_status[[
    'reserve', 'block_num', 'index',  
    'liquidity_rate', 'stable_borrow_rate', 'variable_borrow_rate', 
    'liquidity_index','variable_borrow_index'
]].copy()
liquidation_df = liquidation_df[[
    'block_num', 'index', 'on_behalf_of', 
    'collateral_asset', 'debt_asset', 'debt_to_cover', 'liquidated_collateral_amount',
    'liquidator', 'receive_atoken']].copy()

combine_block_n_index = lambda x: int(str(x['block_num']) + str(x['index']).zfill(6))

interaction_df['block_n_index'] = interaction_df.apply(combine_block_n_index, axis=1)
reserves_status['block_n_index'] = reserves_status.apply(combine_block_n_index, axis=1)
liquidation_df['block_n_index'] = liquidation_df.apply(combine_block_n_index, axis=1)

interaction_df = interaction_df.sort_values('block_n_index').reset_index(drop=True)
reserves_status = reserves_status.sort_values('block_n_index').reset_index(drop=True)


# just give a random reserve address, will be swich in the following part
for index in interaction_df.index:
    if interaction_df.loc[index, 'action'] == "LiquidationCall":
        interaction_df.loc[index, 'reserve'] = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'

# merge
user_df = interaction_df.merge(reserves_status, on=['reserve'], how='left')
change_token_address_to_name = lambda x: revert_token_dict[x] if x in revert_token_dict else x
interaction_df['reserve'] = interaction_df['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
user_df['reserve'] = user_df['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
reserves_status['reserve'] = reserves_status['reserve'].apply(change_token_address_to_name).reset_index(drop=True)

liquidation_df['collateral_asset'] = liquidation_df['collateral_asset'].apply(change_token_address_to_name)
liquidation_df['debt_asset'] = liquidation_df['debt_asset'].apply(change_token_address_to_name)

In [104]:
def get_liquidation_data(df_row):
    df_row = df_row.copy()
    if df_row['action'] != 'LiquidationCall': return df_row
    # collateral
    block_n_index_x = df_row['block_n_index_x']
    liquidation_row = liquidation_df[liquidation_df['block_n_index'] == block_n_index_x]
    collateral_asset = liquidation_row['collateral_asset'].values[0]
    tmp_reserves_status = reserves_status[\
        (reserves_status['reserve'] == collateral_asset) &\
        (reserves_status['block_n_index'] <= block_n_index_x)].copy().sort_values('block_n_index')
    tmp_reserves_status = tmp_reserves_status.iloc[-1, :]

    df_row['block_num_y'] = tmp_reserves_status['block_num']
    df_row['index_y'] = tmp_reserves_status['index']
    df_row['liquidity_rate'] = tmp_reserves_status['liquidity_rate']
    df_row['liquidity_index'] = tmp_reserves_status['liquidity_index']
    df_row['block_n_index_y'] = tmp_reserves_status['block_n_index']

    debt_asset = liquidation_row['debt_asset'].values[0]
    tmp_reserves_status = reserves_status[\
        (reserves_status['reserve'] == debt_asset) &\
        (reserves_status['block_n_index'] <= block_n_index_x)].copy().sort_values('block_n_index')
    tmp_reserves_status = tmp_reserves_status.iloc[-1, :]

    df_row['stable_borrow_rate'] = tmp_reserves_status['stable_borrow_rate']
    df_row['variable_borrow_rate'] = tmp_reserves_status['variable_borrow_rate']
    df_row['variable_borrow_index'] = tmp_reserves_status['variable_borrow_index']
    
    return df_row
    # df_row['block_n_index_x']

In [105]:
# y = reserve update, x = interaction
from_df = user_df[user_df['block_n_index_y'] <= user_df['block_n_index_x']]
from_df = from_df.loc[from_df.groupby('block_n_index_x').block_n_index_y.idxmax()].reset_index(drop=True)
from_df = from_df.apply(get_liquidation_data, axis=1)
# # x = interaction, y = reserve update
# to_df = user_df[user_df['block_n_index_x'] <= user_df['block_n_index_y']]
# to_df = to_df.loc[to_df.groupby('block_n_index_x').block_n_index_y.idxmin()]

print(interaction_df.shape, reserves_status.shape)
from_df.shape #, to_df.shape

(3, 9) (407060, 9)


(3, 17)

In [106]:
from_df

,action,block_num_x,index_x,on_behalf_of,reserve,amount,rate_mode,rate,block_n_index_x,block_num_y,index_y,liquidity_rate,stable_borrow_rate,variable_borrow_rate,liquidity_index,variable_borrow_index,block_n_index_y
0,Deposit,13883845,122,0x39057008ca34c07eefa5bfe78a27c46db2bd49a2,weth,1.500000e+17,-1,-1.000000e+00,13883845000122,13883845,116,6.071552e+22,3.348044e+25,2.784354e+24,1.007696e+27,1.009965e+27,13883845000116
1,Borrow,13885135,460,0x39057008ca34c07eefa5bfe78a27c46db2bd49a2,usdt,5.035837e+08,2,3.423295e+25,13885135000460,13885135,458,2.458863e+25,1.171165e+26,3.423295e+25,1.076012e+27,1.115222e+27,13885135000458
2,LiquidationCall,15094461,45,0x39057008ca34c07eefa5bfe78a27c46db2bd49a2,usdc,-1.000000e+00,-1,-1.000000e+00,15094461000045,15094461,38,7.855859e+24,1.142091e+26,2.841817e+25,1.009419e+27,1.135215e+27,15094461000038


In [117]:
# class TokenInsufficient(Exception):
#     pass


# a token
collateral_dict = defaultdict(float)
collatearl_able_dict = defaultdict(lambda :True)
variable_debt_dict = defaultdict(float)
stable_debt_dict = defaultdict(lambda : [None, None, None]) # amount, interest, start time

liquidation_pool_target = [
    "ReserveUsedAsCollateralEnabled",
    "ReserveUsedAsCollateralDisabled", 
    "Deposit", 
    "Withdraw",
    "Borrow",
    "Repay",
    # "LiquidationCall",
    "Swap",
]

SECONDS_PER_YEAR = 365 * 24 * 60 * 60
RAY = 1e27

ray_mul = lambda a,b: (a * b + RAY/2) / RAY
ray_div = lambda a,b: (a * RAY + b/2) / b

until_block_num = 15180100
sub_interaction_df = interaction_df[interaction_df['block_num'] <= until_block_num].copy()

__library = cdll.LoadLibrary('../eth_crawler/library.so')

get_single_block_time = __library.get_single_block_time
get_single_block_time.argtypes = [c_char_p, GoInt]
get_single_block_time.restype = c_char_p

# Block Time
def get_block_time(block_num):
    try:
        res = get_single_block_time(
            archive_node.encode(), 
            GoInt(int(block_num))
        )
        res = res.decode("utf-8")
        res = json.loads(s=res)#.items()#, columns=['BlockNum', 'Timestamp'])
        
        return res[str(block_num)]
    except Exception as e: 
        print(e)


def cal_stable_debt_change(stable_debt_amount_p, stable_borrow_rate_p, block_num, block_num_p):
    block_time = get_block_time(block_num)
    block_time_p = get_block_time(block_num_p)
    exp = block_time - block_time_p
    
    ###### Reference #####: https://etherscan.io/address/0xc6845a5c768bf8d7681249f8927877efda425baf#code
    expMinusOne = exp - 1
    expMinusTwo = exp - 2 if exp > 2 else 0
    ratePerSecond = stable_borrow_rate_p / SECONDS_PER_YEAR
    basePowerTwo = ray_mul(ratePerSecond, ratePerSecond) # (ratePerSecond * ratePerSecond + 0.5 * RAY)/RAY
    basePowerThree = ray_mul(basePowerTwo, ratePerSecond)#  + 0.5 * RAY)/RAY
    secondTerm = (exp * expMinusOne * basePowerTwo) / 2
    thirdTerm = (exp * expMinusOne * expMinusTwo * basePowerThree) / 6
    compounded_interest = RAY + (ratePerSecond * exp) + (secondTerm) + (thirdTerm)
    new_stable_balance = ray_mul(stable_debt_amount_p, compounded_interest)
    balance_increase = new_stable_balance - stable_debt_amount_p
    ########################################################################

    return new_stable_balance, balance_increase

def update_target_debt_data(action_i, block_num, amount_i, token_name_i, 
        rate_mode_i, liquidity_index, variable_borrow_index, stable_borrow_rate):
    
    
    # ['int', 'liquidityRate'],  # 存钱利息
    # ['int', 'stableBorrowRate'],  # 固定贷款利息
    # ['int', 'variableBorrowRate'], # 可变贷款利息
    # ['int', 'liquidityIndex'], # 存钱token价值指数
    # ['int', 'variableBorrowIndex'], # 可变贷款token价值指数
    # stable = 1, variable = 2
    
    a_token_amount_i = ray_div(amount_i, liquidity_index)
    
    variable_debt_amount_i = ray_div(amount_i, variable_borrow_index)

    # For Stable Debt
    stable_debt_amount_i = amount_i #/ stable_borrow_rate
    stable_debt_amount_p, stable_borrow_rate_p, block_num_p = stable_debt_dict[token_name_i]
    if stable_debt_amount_p != None:
        new_stable_balance, balance_increase = cal_stable_debt_change(stable_debt_amount_p, stable_borrow_rate_p, block_num, block_num_p)

    # block_time = get_block_time(block_num)
    # block_time_p = get_block_time(block_num_p)
    # exp = block_time - block_time_p
    
    # ###### Reference #####: https://etherscan.io/address/0xc6845a5c768bf8d7681249f8927877efda425baf#code
    # expMinusOne = exp - 1
    # expMinusTwo = exp - 2 if exp > 2 else 0
    # ratePerSecond = stable_borrow_rate_p / SECONDS_PER_YEAR
    # basePowerTwo = ray_mul(ratePerSecond, ratePerSecond) # (ratePerSecond * ratePerSecond + 0.5 * RAY)/RAY
    # basePowerThree = ray_mul(basePowerTwo, ratePerSecond)#  + 0.5 * RAY)/RAY
    # secondTerm = (exp * expMinusOne * basePowerTwo) / 2
    # thirdTerm = (exp * expMinusOne * expMinusTwo * basePowerThree) / 6
    # compounded_interest = RAY + (ratePerSecond * exp) + (secondTerm) + (thirdTerm)
    # new_stable_balance = ray_mul(stable_debt_amount_p, compounded_interest)
    # balance_increase = new_stable_balance - stable_debt_amount_p
    # ########################################################################
    
    if action_i == "ReserveUsedAsCollateralEnabled":
        collatearl_able_dict[token_name_i] = True
    elif action_i == "ReserveUsedAsCollateralDisabled":
        collatearl_able_dict[token_name_i] = False
    elif action_i == "Deposit":
        if collatearl_able_dict[token_name_i] == False and collateral_dict[token_name_i] == 0:
            collatearl_able_dict[token_name_i] = True
        collateral_dict[token_name_i] += a_token_amount_i
    elif action_i == 'Withdraw':
        if (collateral_dict[token_name_i] - a_token_amount_i) < 0:
            return False, np.abs(collateral_dict[token_name_i] - a_token_amount_i)
        collateral_dict[token_name_i] -= a_token_amount_i
    elif action_i == "Borrow":
        if rate_mode_i == '1': # stable
            if stable_debt_dict[token_name_i][0] is None:
                stable_debt_dict[token_name_i] = [stable_debt_amount_i, stable_borrow_rate, block_num]
            else:
                stable_debt_dict[token_name_i] = [new_stable_balance + stable_debt_amount_i, stable_borrow_rate, block_num]
        elif rate_mode_i == '2': # variable
            variable_debt_dict[token_name_i] += variable_debt_amount_i
        else:
            assert False, "rate_mode_i error"
    elif action_i == "Repay":
        if rate_mode_i == '1':
            if (new_stable_balance - stable_debt_amount_i) < 0:
                return False, np.abs(new_stable_balance - stable_debt_amount_i)
            stable_debt_dict[token_name_i] = [new_stable_balance - stable_debt_amount_i, stable_borrow_rate, block_num]
        elif rate_mode_i == '2': # variable
            if (variable_debt_dict[token_name_i] - variable_debt_amount_i) < 0:
                return False, np.abs(variable_debt_dict[token_name_i] - variable_debt_amount_i)
            variable_debt_dict[token_name_i] -= variable_debt_amount_i
    else:
        assert False, "Interaction Data error"

    return True, 0

def get_token_value(block_num, index):
    # collateral_dict = defaultdict(float)
    # collatearl_able_dict = defaultdict(lambda :True)
    # variable_debt_dict = defaultdict(float)
    # stable_debt_dict = defaultdict(lambda : [None, None, None])
    collateral_in_original_unit = defaultdict(float)
    var_debt_in_original_unit = defaultdict(float)
    sta_debt_in_original_unit = defaultdict(float)

    block_n_index = combine_block_n_index(dict(block_num=block_num, index=index))

    for token_name, able in collatearl_able_dict.items():
        if able:
            tmp_status = reserves_status[(reserves_status['reserve'] == token_name) &\
                 (reserves_status['block_n_index'] <= block_n_index)].copy().sort_values('block_n_index').iloc[-1,:]
            collateral_in_original_unit[token_name] = ray_mul(collateral_dict[token_name], tmp_status["liquidity_index"])
    
    for token_name, able in variable_debt_dict.items():
        tmp_status = reserves_status[(reserves_status['reserve'] == token_name) &\
                 (reserves_status['block_n_index'] <= block_n_index)].copy().sort_values('block_n_index').iloc[-1,:]
        var_debt_in_original_unit[token_name] = ray_mul(variable_debt_dict[token_name], tmp_status["variable_borrow_index"])

    for token_name, stable_debt in stable_debt_dict.items():
        if stable_debt[0] is not None:
            stable_debt_amount_p, stable_borrow_rate_p, block_num_p = stable_debt_dict[token_name]
            new_stable_balance, balance_increase = cal_stable_debt_change(stable_debt_amount_p, stable_borrow_rate_p, block_num, block_num_p)
            sta_debt_in_original_unit[token_name] = new_stable_balance
    
    return collateral_in_original_unit, var_debt_in_original_unit, sta_debt_in_original_unit


for index_i in sub_interaction_df.index:

    action_i = sub_interaction_df.loc[index_i, 'action']
    block_n_index = sub_interaction_df.loc[index_i, 'block_n_index']
    block_num = sub_interaction_df.loc[index_i, 'block_num']
    index = sub_interaction_df.loc[index_i, 'index']

    # block_time = get_block_time(block_num)
    before_data = from_df[from_df['block_n_index_x'] == block_n_index]#['amount'].values[0]
    liquidity_index = before_data['liquidity_index'].values[0]
    variable_borrow_index = before_data['variable_borrow_index'].values[0]
    stable_borrow_rate = before_data['stable_borrow_rate'].values[0]
    
    if action_i == "LiquidationCall":
        'collateral_asset', 'debt_asset', 'debt_to_cover', 'liquidated_collateral_amount',
        liquidation_i = liquidation_df[liquidation_df['block_n_index'] == block_n_index].copy()
        collateral_asset = liquidation_i.loc[0, 'collateral_asset']
        debt_asset = liquidation_i.loc[0, 'debt_asset']
        debt_to_cover = liquidation_i.loc[0, 'debt_to_cover']
        liquidated_collateral_amount = liquidation_i.loc[0, 'liquidated_collateral_amount']

        # a_token_amount_i = ray_div(liquidated_collateral_amount, liquidity_index)
        # collateral_dict[collateral_asset] -= a_token_amount_i

        collateral_in_original_unit, var_debt_in_original_unit, sta_debt_in_original_unit = get_token_value(block_num, index)

        if var_debt_in_original_unit[debt_asset] < debt_to_cover:
            var_debt_to_liquidate = var_debt_in_original_unit[debt_asset]
            sta_debt_to_repay = debt_to_cover - var_debt_to_liquidate
        else:
            var_debt_to_liquidate = debt_to_cover
            sta_debt_to_repay = 0

        success, remaining_token = update_target_debt_data(
            "Repay", block_num, var_debt_to_liquidate, debt_asset, 
            "2", liquidity_index, variable_borrow_index, stable_borrow_rate)
        assert success

        if sta_debt_to_repay > 0:
            success, remaining_token = update_target_debt_data(
                "Repay", block_num, sta_debt_to_repay, debt_asset, 
                "1", liquidity_index, variable_borrow_index, stable_borrow_rate)
            assert success
        
        success, remaining_token = update_target_debt_data(
                "Withdraw", block_num, liquidated_collateral_amount, collateral_asset, 
                "-1", liquidity_index, variable_borrow_index, stable_borrow_rate)
        assert success

    else:
        amount_i = sub_interaction_df.loc[index_i, 'amount']
        token_name_i = sub_interaction_df.loc[index_i, 'reserve']
        rate_mode_i = sub_interaction_df.loc[index_i, 'rate_mode']

        update_target_debt_data(action_i, block_num, amount_i, token_name_i, 
        rate_mode_i, liquidity_index, variable_borrow_index, stable_borrow_rate)










        
        # ['int', 'liquidityRate'],  # 存钱利息
        # ['int', 'stableBorrowRate'],  # 固定贷款利息
        # ['int', 'variableBorrowRate'], # 可变贷款利息
        # ['int', 'liquidityIndex'], # 存钱token价值指数
        # ['int', 'variableBorrowIndex'], # 可变贷款token价值指数
        # stable = 1, variable = 2
        

        # a_token_amount_i = ray_div(amount_i, liquidity_index)
        
        # variable_debt_amount_i = ray_div(amount_i, variable_borrow_index)

        
        # # For Stable Debt
        # stable_debt_amount_i = amount_i #/ stable_borrow_rate
        # stable_debt_amount_p, stable_borrow_rate_p, block_num_p = stable_debt_dict[token_name_i]
        # block_time_p = get_block_time(block_num_p)
        # exp = block_time - block_time_p
        
        # ###### Reference #####: https://etherscan.io/address/0xc6845a5c768bf8d7681249f8927877efda425baf#code
        # expMinusOne = exp - 1
        # expMinusTwo = exp - 2 if exp > 2 else 0
        # ratePerSecond = stable_borrow_rate_p / SECONDS_PER_YEAR
        # basePowerTwo = ray_mul(ratePerSecond, ratePerSecond) # (ratePerSecond * ratePerSecond + 0.5 * RAY)/RAY
        # basePowerThree = ray_mul(basePowerTwo, ratePerSecond)#  + 0.5 * RAY)/RAY
        # secondTerm = (exp * expMinusOne * basePowerTwo) / 2
        # thirdTerm = (exp * expMinusOne * expMinusTwo * basePowerThree) / 6
        # compounded_interest = RAY + (ratePerSecond * exp) + (secondTerm) + (thirdTerm)
        # new_stable_balance = ray_mul(stable_debt_amount_p, compounded_interest)
        # balance_increase = new_stable_balance - stable_debt_amount_p
        # ########################################################################
        
        # if action_i == "ReserveUsedAsCollateralEnabled":
        #     collatearl_able_dict[token_name_i] = True
        # elif action_i == "ReserveUsedAsCollateralDisabled":
        #     collatearl_able_dict[token_name_i] = False
        # elif action_i == "Deposit":
        #     if collatearl_able_dict[token_name_i] == False and collateral_dict[token_name_i] == 0:
        #         collatearl_able_dict[token_name_i] = True
        #     collateral_dict[token_name_i] += a_token_amount_i
        # elif action_i == 'Withdraw':
        #     collateral_dict[token_name_i] -= a_token_amount_i
        # elif action_i == "Borrow":
        #     if rate_mode_i == '1': # stable
        #         if stable_debt_dict[token_name_i][0] is None:
        #             stable_debt_dict[token_name_i] = [stable_debt_amount_i, stable_borrow_rate, block_num]
        #         else:
        #             stable_debt_dict[token_name_i] = [new_stable_balance + stable_debt_amount_i, stable_borrow_rate, block_num]
        #     elif rate_mode_i == '2': # variable
        #         variable_debt_dict[token_name_i] += variable_debt_amount_i
        #     else:
        #         assert False, "rate_mode_i error"
        # elif action_i == "Repay":
        #     if rate_mode_i == '1':
        #         stable_debt_dict[token_name_i] = [new_stable_balance - stable_debt_amount_i, stable_borrow_rate, block_num]
        #     elif rate_mode_i == '2': # variable
        #         variable_debt_dict[token_name_i] -= variable_debt_amount_i
        # else:
        #     assert False, sub_interaction_df.loc[index_i, :]
            
# print("Collatearl: ", collateral_dict)
# print("Debt: ", debt_dict)

In [118]:
collateral_dict


defaultdict(float, {'weth': 705258282624.0})

In [119]:
collatearl_able_dict

defaultdict(<function __main__.<lambda>()>, {'weth': True})

In [120]:
variable_debt_dict

defaultdict(float, {'usdt': 303660442.80859745})

In [121]:
stable_debt_dict

defaultdict(<function __main__.<lambda>()>,
            {'weth': [None, None, None], 'usdt': [None, None, None]})

In [122]:
get_token_value(until_block_num, 10000)

(defaultdict(float, {'weth': 712119309038.4827}),
 defaultdict(float, {'usdt': 345069712.92749935}),
 defaultdict(float, {}))

In [16]:
sub_interaction_df.loc[index_i, 'block_n_index']

1465568743

In [123]:
345069712 / 1850

186524.16864864866

712515711904
712119309038.4827
345069712 / 1850

In [17]:
sub_interaction_df

,action,block_num,index,on_behalf_of,reserve,amount,rate_mode,rate,block_n_index
0,LiquidationCall,14655687,43,0x3dd3d4db709ddf8b18d05e1ca868b141aa7962e5,-1,-1.000000e+00,-1,-1.000000e+00,1465568743
1,Deposit,14650571,217,0x3dd3d4db709ddf8b18d05e1ca868b141aa7962e5,usdc,3.605526e+10,-1,-1.000000e+00,14650571217
2,Borrow,14650571,228,0x3dd3d4db709ddf8b18d05e1ca868b141aa7962e5,weth,1.040754e+19,2,1.446617e+25,14650571228
3,Repay,14680475,228,0x3dd3d4db709ddf8b18d05e1ca868b141aa7962e5,weth,5.205322e+18,-1,-1.000000e+00,14680475228
4,Withdraw,14680475,241,0x3dd3d4db709ddf8b18d05e1ca868b141aa7962e5,usdc,1.976619e+10,-1,-1.000000e+00,14680475241


In [203]:
collatearl_able_dict[0]

True

In [ ]:
reserves_status.columns

In [ ]:

reserves_status

,value
key,
blockHash,0xdc69d208ffb9758a97b8a5a6bcff9682766c2ff40e5e...
blockNumber,0xdfa0c7
from,0x3b794929566e3ba0f25e4263e1987828b5c87161
gas,0xc350
gasPrice,0x1a944cfab4
hash,0xdba8e0e5a5603c5dffb452f1f05d613942a2f253db56...
input,0x
nonce,0x5dd1b
to,0x987a3a7e99ee7d463c2fbddc8e0c1a6280bb1e4f


In [46]:
df.loc["hash"].values[0]

'0xdba8e0e5a5603c5dffb452f1f05d613942a2f253db565cffd285823a60cd48b1'